In [2]:
#カテゴリ特徴量のターゲット符号化。

#カテゴリターゲットの場合：特徴量は，
#特定のカテゴリ値が与えられたターゲットの事後確率と，
#すべての学習データにおけるターゲットの事後確率のブレンドで置き換えられます．

#連続ターゲットの場合：特徴量は、特定のカテゴリ値が与えられたターゲットの期待値と、
#すべての学習データにおけるターゲットの期待値のブレンドで置き換えられます。

# ・Count Encoding 登場回数のカウント
# ・Target Encoding SUM(y)/登場回数
# 　※Leakになるのでtestデータにはfit厳禁
# ・Leave-one-out Encoding その前の行までのSUM(y)/登場回数　
# 　※要シャッフル。Leakになるのでtestデータにはfit厳禁
# ・CatBoost Encoding　その前の行までのSUM(y)/登場回数＋補正
# 　※要シャッフル。Leakになるのでtestデータにはfit厳禁
# ・Feature embedding with SVD 

In [1]:
import numpy as np
import pandas as pd              # version 0.23.4
import category_encoders as ce   # version 1.2.8

pd.options.display.float_format = '{:.2f}'.format # to make legible

# some new data frame for the contrast encoders
df3 = pd.DataFrame(
    {'color':[3,2,0,1,1,1,2,2,3,3,4,5,6,7,8,3], 
     'outcome':[1,0,0,1,2,1,1,0,0,1,0,2,2,1,1,1]})

# set up X and y
X3 = df3.drop('outcome', axis = 1)
y3 = df3.drop('color', axis = 1)

df3

,color,outcome
0,3,1
1,2,0
2,0,0
3,1,1
4,1,2
5,1,1
6,2,1
7,2,0
8,3,0
9,3,1


In [9]:
# Target with default parameters
ce_target = ce.TargetEncoder(cols = ['color'])

ce_target.fit(X3, y3['outcome'])
# Must pass the series for y in v1.2.8

df3["ce"] = ce_target.transform(X3, y3['outcome'])
df3.sort_values("color")

,color,outcome,ce
2,0,0,0.88
3,1,1,1.28
4,1,2,1.28
5,1,1,1.28
1,2,0,0.40
6,2,1,0.40
7,2,0,0.40
0,3,1,0.76
8,3,0,0.76
9,3,1,0.76


In [12]:

# Target with default parameters
ce_target = ce.TargetEncoder(cols = ['color'],smoothing=0.001)

ce_target.fit(X3, y3['outcome'])
# Must pass the series for y in v1.2.8

df3["ce"] = ce_target.transform(X3, y3['outcome'])
df3.sort_values("color")

,color,outcome,ce
2,0,0,0.88
3,1,1,1.33
4,1,2,1.33
5,1,1,1.33
1,2,0,0.33
6,2,1,0.33
7,2,0,0.33
0,3,1,0.75
8,3,0,0.75
9,3,1,0.75


In [26]:
print( df3[df3["color"]==1]["outcome"].sum() / len(df3[df3["color"]==1]) )
print( df3[df3["color"]==1]["ce"] )

1.3333333333333333
3   1.33
4   1.33
5   1.33
Name: ce, dtype: float64


In [27]:
i=2
print( df3[df3["color"]==i]["outcome"].sum() / len(df3[df3["color"]==i]) )
print( df3[df3["color"]==i]["ce"] )

0.3333333333333333
1   0.33
6   0.33
7   0.33
Name: ce, dtype: float64


In [28]:
i=3
print( df3[df3["color"]==i]["outcome"].sum() / len(df3[df3["color"]==i]) )
print( df3[df3["color"]==i]["ce"] )

0.75
0    0.75
8    0.75
9    0.75
15   0.75
Name: ce, dtype: float64


In [46]:
# 0, 4,5,6,7,8 は1回しか登場しないので、なんか補正されるのかな？何の値かはちょっとわからん
i=0
print( df3[df3["color"]==i]["outcome"].sum() / len(df3[df3["color"]==i]) )
print( df3[df3["color"]==i]["ce"] )

0.0
2   0.88
Name: ce, dtype: float64


In [29]:
i=4
print( df3[df3["color"]==i]["outcome"].sum() / len(df3[df3["color"]==i]) )
print( df3[df3["color"]==i]["ce"] )

0.0
10   0.88
Name: ce, dtype: float64


In [30]:
i=5
print( df3[df3["color"]==i]["outcome"].sum() / len(df3[df3["color"]==i]) )
print( df3[df3["color"]==i]["ce"] )

2.0
11   0.88
Name: ce, dtype: float64


In [31]:
i=6
print( df3[df3["color"]==i]["outcome"].sum() / len(df3[df3["color"]==i]) )
print( df3[df3["color"]==i]["ce"] )

2.0
12   0.88
Name: ce, dtype: float64


In [32]:
i=7
print( df3[df3["color"]==i]["outcome"].sum() / len(df3[df3["color"]==i]) )
print( df3[df3["color"]==i]["ce"] )

1.0
13   0.88
Name: ce, dtype: float64


In [33]:
i=8
print( df3[df3["color"]==i]["outcome"].sum() / len(df3[df3["color"]==i]) )
print( df3[df3["color"]==i]["ce"] )

1.0
14   0.88
Name: ce, dtype: float64


In [49]:
print( df3[ (df3["color"]>=4) | (df3["color"]==0) ]["outcome"].sum() / len(df3[ (df3["color"]>=4) | (df3["color"]==0) ]) )

1.0


In [50]:
df3[ (df3["color"]>=4) | (df3["color"]==0) ]

,color,outcome,ce
2,0,0,0.88
10,4,0,0.88
11,5,2,0.88
12,6,2,0.88
13,7,1,0.88
14,8,1,0.88


In [52]:
6*0.88, 4*0.88

(5.28, 3.52)

In [56]:
(6+2) / 6

1.3333333333333333

In [57]:
(6+2) / 12

0.6666666666666666